In [1]:
import numpy as np
from scipy import sparse

class ReducedRankRegressor(object):
    """
    Reduced Rank Regressor (linear 'bottlenecking' or 'multitask learning')
    - X is an n-by-d matrix of features.
    - Y is an n-by-D matrix of targets.
    - rrank is a rank constraint.
    - reg is a regularization parameter (optional).
    """
    def __init__(self, X, Y, rank, reg=None):
        if np.size(np.shape(X)) == 1:
            X = np.reshape(X, (-1, 1))
        if np.size(np.shape(Y)) == 1:
            Y = np.reshape(Y, (-1, 1))
        if reg is None:
            reg = 0
        self.rank = rank

        CXX = np.dot(X.T, X) + reg * sparse.eye(np.size(X, 1))
        CXY = np.dot(X.T, Y)
        _U, _S, V = np.linalg.svd(np.dot(CXY.T, np.dot(np.linalg.pinv(CXX), CXY)))
        self.W = V[0:rank, :].T
        self.A = np.dot(np.linalg.pinv(CXX), np.dot(CXY, self.W)).T

    def __str__(self):
        return 'Reduced Rank Regressor (rank = {})'.format(self.rank)

    def predict(self, X):
        """Predict Y from X."""
        if np.size(np.shape(X)) == 1:
            X = np.reshape(X, (-1, 1))
        return np.dot(X, np.dot(self.A.T, self.W.T))

In [2]:
import os
import numpy as np
from EnsemblePursuit.EnsemblePursuit import EnsemblePursuit
import matplotlib.pyplot as plt

In [3]:
fname='/media/maria/DATA1/Documents/NeuroMatchAcademy2020_dat/kay_images.npz'
with np.load(fname) as dobj:
    dat = dict(**dobj)

In [4]:
def get_rois(dat,ind):
    spec_rois=np.where(dat['roi']==ind)
    #print(spec_rois)
    spec_dat=dat['responses'][:,spec_rois].reshape(1750,-1)
    return spec_dat, spec_rois

def rrregression(dat,inds):
    spec_dat0,spec_roi0=get_rois(dat,inds[0])
    spec_dat1,spec_roi1=get_rois(dat,inds[1])
    rrr=ReducedRankRegressor(spec_dat0,spec_dat1,rank=30)
    test0=dat['responses_test'][:,spec_roi0].reshape(120,-1)
    test1=dat['responses_test'][:,spec_roi1].reshape(120,-1)
    pred=rrr.predict(test0)
    corr=np.corrcoef(test1.flatten(),pred.flatten())[0,1]
    return corr

In [5]:
corr_lst=[]
for j in range(1,8):
    corr_row=[]
    for i in range(1,8):
        if i!=j:
            cor=rrregression(dat,[i,j])
        if i==j:
            cor=0
        print(cor)    
        corr_row.append(cor)
    corr_lst.append(corr_row)
    
corr_lst=np.array(corr_lst)
sns.heatmap(corr_lst)

0
0.46836761281619904
0.21446256961060192
0.3202931658369222
0.29301466938110643
0.263071894451033
0.313437061578201
0.4632887957426442
0
0.24005721049788634
0.32673856258022227
0.3181393317955485
0.3174660168680035
0.3198082933857296
0.3445442283688504
0.3879358658726048
0
0.33327360424178337
0.33204928065514305
0.30909914558694984
0.3347561959902074
0.1624027766748163
0.23985563412180105
0.10432645287690283
0
0.3918505822769027
0.19557888940832943
0.31301248357141614
0.13311318497083022
0.17840942692601408
0.1054477700532839
0.4110824154934709
0
0.21135613511020518
0.31181689776661464
0.25426192628243827
0.30302713670330106
0.1616043092897154
0.2987005483069137
0.30424377000118646
0
0.35810384313625465
0.24296447002634422
0.2967488820848376
0.09205770192716611
0.38368494875377457
0.3934574908744777
0.32794180899599534
0


NameError: name 'sns' is not defined

In [6]:

sns.heatmap(corr_lst)

NameError: name 'sns' is not defined

In [5]:
dat.keys()

dict_keys(['stimuli', 'stimuli_test', 'responses', 'responses_test', 'roi', 'roi_names'])

In [6]:
print(dat['roi_names'])

['Other' 'V1' 'V2' 'V3' 'V3A' 'V3B' 'V4' 'LatOcc']


In [7]:
print(dat['responses'].shape)

(1750, 8428)


In [26]:
v1_rois=np.where(dat['roi']==1)
print(v1_rois)
v1_dat=dat['responses'][:,v1_rois].reshape(1750,-1)

(array([ 187,  199,  213, ..., 8281, 8331, 8355]),)


In [33]:
v2_rois=np.where(dat['roi']==2)
print(v2_rois)
v2_dat=dat['responses'][:,v2_rois].reshape(1750,-1)

(array([  26,   27,   29, ..., 8381, 8399, 8400]),)


In [86]:
rrr=ReducedRankRegressor(v1_dat,v2_dat,rank=40)

In [87]:
v1_test=dat['responses_test'][:,v1_rois].reshape(120,-1)
v2_test=dat['responses_test'][:,v2_rois].reshape(120,-1)

In [ ]:
pred=rrr.predict(v1_test)

In [54]:
print(pred.shape)
print(v2_test.shape)

(120, 2083)
(120, 2083)


In [88]:
print(np.corrcoef(v2_test.flatten(),pred.flatten()))

[[1.         0.45449689]
 [0.45449689 1.        ]]


In [34]:
ep=EnsemblePursuit(lam=0.01,n_components=100,n_kmeans=100)
ep.fit(dat['responses'])

obtained 100 PCs in 5.6839 seconds
initialized 100 clusters with k-means in 2.0784 seconds
ensemble 0, time 3.22, nr neurons 5040, EV 0.0630
ensemble 25, time 26.12, nr neurons 634, EV 0.1846
ensemble 50, time 41.30, nr neurons 369, EV 0.2195
ensemble 75, time 52.70, nr neurons 201, EV 0.2394
ensemble 99, time 63.00, nr neurons 142, EV 0.2544
average sparsity is 0.0678


In [37]:
U=ep.weights
ensemble=np.nonzero(U[:,0])[0]

In [ ]:
corr[corr>0.5]

In [44]:
n_ens=set(ensemble)
v1_overlap=list(n_ens.intersection(set(list(v1_rois[0].flatten()))))
v2_overlap=list(n_ens.intersection(set(list(v2_rois[0].flatten()))))

In [48]:
print(len(v1_overlap))

920


In [47]:
print(v2_rois[0].shape)

(2083,)


In [49]:
print(corr.shape)

(240, 240)
